<a href="https://colab.research.google.com/github/Legajo/Colab-Notebooks/blob/main/HW2_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# ===============================================================================
# COMPLETE IPO SHARPE RATIO ANALYSIS - ALL PROMPTS IN SEQUENCE
# ===============================================================================
# Question: What is the median Sharpe ratio (as of June 6, 2025) for companies
#           that went public in the first 5 months of 2024?
# ===============================================================================

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import yfinance as yf
import time

# ===============================================================================
# PROMPT 1: SCRAPE 2024 IPOs AND FILTER TO FIRST 5 MONTHS
# ===============================================================================

print("\n" + "="*80)
print("PROMPT 1: SCRAPING IPO DATA")
print("="*80)

url = 'https://stockanalysis.com/ipos/2024/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

table = soup.find('table')
df_ipos = pd.read_html(str(table))[0]

print(f"✓ Scraped {len(df_ipos)} IPOs from 2024")

# Identify and convert date column
date_col = [col for col in df_ipos.columns if 'date' in col.lower() or 'priced' in col.lower()][0]
df_ipos[date_col] = pd.to_datetime(df_ipos[date_col])

# Filter to first 5 months of 2024
cutoff_date = datetime(2024, 6, 1)
df_filtered = df_ipos[df_ipos[date_col] < cutoff_date].copy()

# Extract ticker symbols
ticker_col = [col for col in df_filtered.columns if 'symbol' in col.lower() or 'ticker' in col.lower()][0]
ALL_TICKERS = df_filtered[ticker_col].tolist()

print(f"✓ Filtered to {len(ALL_TICKERS)} tickers (Expected: 75)")
print(f"✓ Date range: {df_filtered[date_col].min().date()} to {df_filtered[date_col].max().date()}")




PROMPT 1: SCRAPING IPO DATA
✓ Scraped 225 IPOs from 2024
✓ Filtered to 77 tickers (Expected: 75)
✓ Date range: 2024-01-09 to 2024-05-31


/tmp/ipython-input-2495516042.py:33: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_ipos = pd.read_html(str(table))[0]


In [12]:
# ===============================================================================
# PROMPT 2-3: DOWNLOAD DAILY OHLCV DATA USING YFINANCE
# ===============================================================================

print("\n" + "="*80)
print("PROMPT 2-3: DOWNLOADING STOCK DATA")
print("="*80)
print(f"Downloading {len(ALL_TICKERS)} tickers (will take ~{len(ALL_TICKERS)} seconds)")
print("="*80 + "\n")

# Initialize empty DataFrame (Code Snippet 7 pattern)
stocks_df = pd.DataFrame({'A': []})

for i, ticker in enumerate(ALL_TICKERS):
    print(f"{i:2d} {ticker}", end=" ")

    try:
        ticker_obj = yf.Ticker(ticker)
        historyPrices = ticker_obj.history(period="max", interval="1d")

        if historyPrices.empty:
            print("⚠ NO DATA")
            continue

        # Generate features
        historyPrices['Ticker'] = ticker
        historyPrices['Year'] = historyPrices.index.year
        historyPrices['Month'] = historyPrices.index.month
        historyPrices['Weekday'] = historyPrices.index.weekday
        historyPrices['Date'] = historyPrices.index.date

        # Historical returns - including growth_252d
        for days in [1, 3, 7, 30, 90, 252, 365]:
            historyPrices[f'growth_{days}d'] = historyPrices['Close'] / historyPrices['Close'].shift(days)

        historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

        # Technical indicators
        historyPrices['SMA10'] = historyPrices['Close'].rolling(10).mean()
        historyPrices['SMA20'] = historyPrices['Close'].rolling(20).mean()
        historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
        historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

        # Volatility formula
        historyPrices['volatility'] = historyPrices['Close'].rolling(30).std() * np.sqrt(252)

        historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

        print(f"✓ ({len(historyPrices)} rows)")

        # Sleep 1 sec between downloads
        time.sleep(1)

        # Concatenate
        if stocks_df.empty:
            stocks_df = historyPrices
        else:
            stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

    except Exception as e:
        print(f"⚠ ERROR: {e}")
        continue

stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

print(f"\n✓ Downloaded {len(stocks_df):,} rows for {stocks_df['Ticker'].nunique()} tickers")



PROMPT 2-3: DOWNLOADING STOCK DATA

 0 NAKA ✓ (340 rows)
 1 BOW ✓ (345 rows)
 2 HDL ✓ (349 rows)
 3 RFAI ✓ (317 rows)
 4 JDZG ✓ (351 rows)
 5 RAY ✓ (351 rows)
 6 BTOC ✓ (352 rows)
 7 ZK ✓ (354 rows)
 8 GPAT ✓ (319 rows)
 9 PAL ✓ (355 rows)
10 SVCO ✓ (355 rows)
11 NNE ✓ (356 rows)
12 CCIX ✓ (326 rows)
13 VIK ✓ (361 rows)
14 ZONE ✓ (364 rows)
15 LOAR ✓ (365 rows)
16 MRX ✓ (365 rows)
17 RBRK ✓ (365 rows)
18 NCI ✓ (367 rows)
19 MFI ✓ (368 rows)
20 YYGH ✓ (368 rows)
21 TRSG ✓ (369 rows)
22 CDTG ✓ (369 rows)
23 CTRI ✓ (369 rows)
24 IBTA ✓ (369 rows)
25 MTEN ✓ (369 rows)
26 SUPX ✓ (371 rows)
27 TWG ✓ (371 rows)
28 ULS ✓ (374 rows)
29 PACS ✓ (375 rows)
30 MNDR ✓ (376 rows)
31 CTNM ✓ (379 rows)
32 MAMO ✓ (382 rows)
33 ZBAO ✓ (382 rows)
34 BOLD ✓ (384 rows)
35 MMA ✓ (384 rows)
36 UBXG ✓ (384 rows)
37 IBAC ✓ (360 rows)
38 AUNA ✓ (388 rows)
39 BKHA ✓ (353 rows)
40 LOBO ✓ (389 rows)
41 RDDT ✓ (389 rows)
42 ALAB ✓ (390 rows)
43 INTJ ✓ (390 rows)
44 RYDE ✓ (400 rows)
45 LGCL ✓ (401 rows)
46 SMXT ✓ (

ERROR:yfinance:$CHRO: possibly delisted; no timezone found


⚠ NO DATA
50 UMAC ✓ (414 rows)
51 HLXB ✓ (376 rows)
52 MGX ✓ (417 rows)
53 TBBB ✓ (417 rows)
54 TELO ✓ (417 rows)
55 KYTX ✓ (417 rows)
56 PMNT ✓ (418 rows)
57 AHR ✓ (419 rows)
58 LEGT ✓ (384 rows)
59 ANRO ✓ (422 rows)
60 GUTS ✓ (422 rows)
61 AS ✓ (423 rows)
62 FBLG ✓ (424 rows)
63 AVBP ✓ (427 rows)
64 BTSG ✓ (427 rows)
65 HAO ✓ (427 rows)
66 CGON ✓ (428 rows)
67 YIBO ✓ (428 rows)
68 JL ✓ (429 rows)
69 SUGP ✓ (429 rows)
70 JVSA 

ERROR:yfinance:$JVSA: possibly delisted; no timezone found


⚠ NO DATA
71 KSPI ✓ (432 rows)
72 CCTG ✓ (433 rows)
73 PSBD ✓ (433 rows)
74 SYNX ✓ (436 rows)
75 SDHC ✓ (437 rows)
76 ROMA ✓ (439 rows)

✓ Downloaded 28,885 rows for 75 tickers


In [13]:

# ===============================================================================
# PROMPT 4: VERIFY DATA
# ===============================================================================

print("\n" + "="*80)
print("PROMPT 4: DATA VERIFICATION")
print("="*80)

print(f"\nUnique tickers: {stocks_df['Ticker'].nunique()}")
print(f"Date range: {stocks_df['Date'].min().date()} to {stocks_df['Date'].max().date()}")

print("\nObservations per ticker (top 10):")
print(stocks_df['Ticker'].value_counts().head(10))

# ===============================================================================
# PROMPT 5: CALCULATE SHARPE RATIO
# ===============================================================================

print("\n" + "="*80)
print("PROMPT 5: CALCULATING SHARPE RATIO")
print("="*80)

# Sharpe = (growth_252d - 0.045) / volatility
stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']

print(f"✓ Sharpe ratio calculated")
print(f"✓ Rows with Sharpe: {stocks_df['Sharpe'].notna().sum():,} / {len(stocks_df):,}")


PROMPT 4: DATA VERIFICATION

Unique tickers: 75
Date range: 2024-01-09 to 2025-10-08

Observations per ticker (top 10):
Ticker
ROMA    439
SDHC    437
SYNX    436
CCTG    433
PSBD    433
KSPI    432
SUGP    429
JL      429
YIBO    428
CGON    428
Name: count, dtype: int64

PROMPT 5: CALCULATING SHARPE RATIO
✓ Sharpe ratio calculated
✓ Rows with Sharpe: 9,985 / 28,885


In [14]:

# ===============================================================================
# PROMPT 6: FILTER TO JUNE 6, 2025
# ===============================================================================

print("\n" + "="*80)
print("PROMPT 6: FILTERING TO JUNE 6, 2025")
print("="*80)

target_date = pd.to_datetime('2025-06-06')
df_june6 = stocks_df[stocks_df['Date'] == target_date].copy()

print(f"✓ Tickers on {target_date.date()}: {len(df_june6)}")
print(f"✓ With growth_252d: {df_june6['growth_252d'].notna().sum()} / {len(df_june6)} (Expected: 71/75)")



PROMPT 6: FILTERING TO JUNE 6, 2025
✓ Tickers on 2025-06-06: 75
✓ With growth_252d: 71 / 75 (Expected: 71/75)


In [15]:

# ===============================================================================
# PROMPT 7: DESCRIPTIVE STATISTICS
# ===============================================================================

print("\n" + "="*80)
print("PROMPT 7: DESCRIPTIVE STATISTICS")
print("="*80)

print("\nGROWTH_252D STATISTICS:")
print("-" * 80)
print(df_june6['growth_252d'].describe())

median_growth = df_june6['growth_252d'].median()
mean_growth = df_june6['growth_252d'].mean()
print(f"\nMedian: {median_growth:.4f} → {(median_growth-1)*100:.1f}% change (Expected: ~-25%)")
print(f"Mean:   {mean_growth:.4f} → {(mean_growth-1)*100:.1f}% change (Expected: ~+15%)")

print("\n" + "-" * 80)
print("SHARPE RATIO STATISTICS:")
print("-" * 80)
print(df_june6['Sharpe'].describe())

positive_sharpe = (df_june6['Sharpe'] > 0).sum()
total_sharpe = df_june6['Sharpe'].notna().sum()
print(f"\nPositive Sharpe (beat 4.5% risk-free): {positive_sharpe} / {total_sharpe}")



PROMPT 7: DESCRIPTIVE STATISTICS

GROWTH_252D STATISTICS:
--------------------------------------------------------------------------------
count    71.000000
mean      1.223972
std       1.505650
min       0.024970
25%       0.286818
50%       0.708330
75%       1.464634
max       8.097413
Name: growth_252d, dtype: float64

Median: 0.7083 → -29.2% change (Expected: ~-25%)
Mean:   1.2240 → 22.4% change (Expected: ~+15%)

--------------------------------------------------------------------------------
SHARPE RATIO STATISTICS:
--------------------------------------------------------------------------------
count    71.000000
mean      0.235039
std       0.470033
min      -0.079677
25%       0.030033
50%       0.058319
75%       0.208247
max       2.835668
Name: Sharpe, dtype: float64

Positive Sharpe (beat 4.5% risk-free): 70 / 71


In [16]:

# ===============================================================================
# PROMPT 8: ANSWER MAIN QUESTION
# ===============================================================================

print("\n" + "="*80)
print("PROMPT 8: FINAL ANSWER")
print("="*80)

median_sharpe = df_june6['Sharpe'].median()

print(f"\n🎯 MEDIAN SHARPE RATIO: {median_sharpe:.4f}")
print(f"\n{'='*80}")
print(f"Based on {df_june6['Sharpe'].notna().sum()} stocks from first 5 months of 2024")
print(f"Measured as of: {target_date.date()}")
print(f"Period: 252 trading days (~1 year)")
print(f"Risk-free rate: 4.5%")
print("="*80)



PROMPT 8: FINAL ANSWER

🎯 MEDIAN SHARPE RATIO: 0.0583

Based on 71 stocks from first 5 months of 2024
Measured as of: 2025-06-06
Period: 252 trading days (~1 year)
Risk-free rate: 4.5%


In [17]:
# ===============================================================================
# PROMPT 9: TOP 10 COMPARISON
# ===============================================================================

print("\n" + "="*80)
print("PROMPT 9: TOP 10 COMPARISON (GROWTH vs SHARPE)")
print("="*80)

df_valid = df_june6.dropna(subset=['growth_252d', 'Sharpe'])

print("\nTOP 10 BY GROWTH_252D:")
print("-" * 80)
top10_growth = df_valid.nlargest(10, 'growth_252d')[['Ticker', 'growth_252d', 'volatility', 'Sharpe']]
top10_growth['Return%'] = (top10_growth['growth_252d'] - 1) * 100
print(top10_growth[['Ticker', 'Return%', 'growth_252d', 'volatility', 'Sharpe']].to_string(index=False))

print("\n" + "-" * 80)
print("TOP 10 BY SHARPE RATIO:")
print("-" * 80)
top10_sharpe = df_valid.nlargest(10, 'Sharpe')[['Ticker', 'Sharpe', 'growth_252d', 'volatility']]
top10_sharpe['Return%'] = (top10_sharpe['growth_252d'] - 1) * 100
print(top10_sharpe[['Ticker', 'Sharpe', 'Return%', 'growth_252d', 'volatility']].to_string(index=False))

# Overlap analysis
growth_set = set(top10_growth['Ticker'].tolist())
sharpe_set = set(top10_sharpe['Ticker'].tolist())
overlap = growth_set.intersection(sharpe_set)

print("\n" + "-" * 80)
print("OVERLAP ANALYSIS:")
print("-" * 80)
print(f"Companies in both top 10: {len(overlap)} / 10")

if len(overlap) == 10:
    print("✓ YES - Same companies (though possibly different order)")
else:
    print("✗ NO - Different companies")
    print(f"\nCommon: {sorted(overlap)}")
    print(f"Only in growth: {sorted(growth_set - sharpe_set)}")
    print(f"Only in Sharpe: {sorted(sharpe_set - growth_set)}")
    print("\n💡 High growth ≠ High Sharpe when volatility is very high")
    print("   Sharpe adjusts for risk - stable growers can beat volatile winners")

print("\n" + "="*80)
print("IPO SHARPE RATIO ANALYSIS COMPLETE")
print("="*80)
print(f"\n✓ stocks_df created with {len(stocks_df):,} rows")
print(f"✓ Ready for Fixed Months Holding Strategy analysis")
print("="*80)


PROMPT 9: TOP 10 COMPARISON (GROWTH vs SHARPE)

TOP 10 BY GROWTH_252D:
--------------------------------------------------------------------------------
Ticker    Return%  growth_252d  volatility   Sharpe
    JL 709.741268     8.097413   14.221305 0.566222
  ROMA 515.640596     6.156406   12.553202 0.486840
  NAKA 443.835606     5.438356  136.860031 0.039408
  UMAC 396.653299     4.966533   11.681615 0.421306
   NNE 365.522409     4.655224   57.122852 0.080707
  RBRK 218.406516     3.184065  164.768721 0.019051
   AHR 148.309701     2.483097   21.378546 0.114044
    AS 147.820341     2.478203   86.819697 0.028026
  SUPX 134.606477     2.346065    8.637976 0.266389
   MRX 130.038436     2.300384   29.345061 0.076857

--------------------------------------------------------------------------------
TOP 10 BY SHARPE RATIO:
--------------------------------------------------------------------------------
Ticker   Sharpe    Return%  growth_252d  volatility
  BKHA 2.835668   4.588063     1.045